In [1]:
import numpy as np
import pandas as pd
from scipy.stats import f, t, chi2
from statsmodels.stats import multivariate as mv

In [2]:
#1
#(a)
lumber_raw = pd.read_fwf('lumber.dat', header=None)
lumber_raw

,0,1
0,1232,4175
1,1115,6652
2,2205,7612
3,1897,10914
4,1932,10850
5,1612,7627
6,1598,6954
7,1804,8365
8,1752,9469
9,2067,6410


In [3]:
lumber1=lumber_raw.iloc[0:15,:]
lumber1.columns=['x1','x2']
lumber2=lumber_raw.iloc[15:,:]
lumber2.reset_index(inplace = True, drop = True)
lumber2.columns=['x1','x2']
lumber=pd.concat([lumber1,lumber2],axis=1)
lumber

,x1,x2,x1,x2
0,1232,4175,1712,7749
1,1115,6652,1932,6818
2,2205,7612,1820,9307
3,1897,10914,1900,6457
4,1932,10850,2426,10102
5,1612,7627,1558,7414
6,1598,6954,1470,7556
7,1804,8365,1858,7833
8,1752,9469,1587,8309
9,2067,6410,2208,9559


In [4]:
lumber_diff=lumber1-lumber2
lumber_diff

,x1,x2
0,-480,-3574
1,-817,-166
2,385,-1695
3,-3,4457
4,-494,748
5,54,213
6,128,-602
7,-54,532
8,165,1160
9,-141,-3149


In [5]:
def mean_vector(data):
    ones = pd.DataFrame({'mean': np.ones(len(data))})
    mean = ones.transpose().dot(data)/len(data)
    return mean

In [6]:
def cov_matrix(data):
    mean = mean_vector(data)
    mean_rep = pd.concat([mean]*len(data))
    mean_rep.columns = data.columns
    mean_rep.reset_index(inplace = True, drop = True)
    cov = (data-mean_rep).transpose().dot(data-mean_rep)/(len(data)-1)
    return cov

In [7]:
#Hotelling's T^2 test
def my_hotelling(df,mu):
    n=len(df.index) #number of observations
    p=len(df.columns) #number of variables
    mu_reshape=pd.DataFrame(mu.reshape(1,len(mu)), index=['mean'], columns=df.columns)
    diff=mean_vector(df)-mu_reshape
    inv_cov = pd.DataFrame(np.linalg.inv(cov_matrix(df)), index=df.columns, columns=df.columns)
    t2 = n*((diff.dot(inv_cov)).dot(diff.T)).iloc[0,0]
    fvalue = (n-p)*t2/((n-1)*p)
    pvalue = f.sf(fvalue, p, n-p)
    return {'t2': t2, 'f-value': fvalue, 'p-value': pvalue}

In [8]:
#array of means to test
d0=np.array([0,0])
d0

array([0, 0])

In [9]:
my_hotelling(lumber_diff,d0)
#cannot reject null hypothesis

{'t2': 2.734908183617629,
 'f-value': 1.2697787995367562,
 'p-value': 0.3135310532015403}

In [10]:
#(b)
def sim_conf_int(df,n,p,alpha):
    d_bar = mean_vector(df).iloc[0,0]
    S2=cov_matrix(df).iloc[0,0]
    term2 = np.sqrt((n-1)*p/(n*(n-p))*f.isf(alpha,p,n-p)*S2)
    interval=[d_bar-term2, d_bar+term2]
    return interval

In [11]:
sim_conf_int(lumber_diff,15,2,0.05)

[-546.0898558358173, 159.2898558358173]

In [12]:
#(c)
def prof_analysis(df,C):
    n = len(df.index)
    p = len(C.index)
    x_bar = mean_vector(df)
    S = cov_matrix(df)
    t2 = n*((C.dot(x_bar.T)).T.dot(np.linalg.inv(C.dot(S).dot(C.T))).dot(C.dot(x_bar.T))).iloc[0,0]
    fvalue = t2*(n-p)/((n-1)*p)
    pvalue = f.sf(fvalue, p, n-p)
    return {'t2': t2, 'f-value': fvalue, 'p-value': pvalue}

In [13]:
lumber

,x1,x2,x1,x2
0,1232,4175,1712,7749
1,1115,6652,1932,6818
2,2205,7612,1820,9307
3,1897,10914,1900,6457
4,1932,10850,2426,10102
5,1612,7627,1558,7414
6,1598,6954,1470,7556
7,1804,8365,1858,7833
8,1752,9469,1587,8309
9,2067,6410,2208,9559


In [14]:
C1 = pd.DataFrame([[1,0,-1,0],[0,1,0,-1]],columns=['x1','x2','x1','x2'])
C1

,x1,x2,x1,x2
0,1,0,-1,0
1,0,1,0,-1


In [15]:
prof_analysis(lumber,C1)
#same result as (a)

{'t2': 2.7349081836176325,
 'f-value': 1.269778799536758,
 'p-value': 0.31353105320153973}

In [16]:
#2
#Hotelling's T^2 test
def two_sample_hotelling(df):
    n1=int(len(df.index)/2) #number of observations for X1
    n2=n1
    p=len(df.columns)-1 #number of variables
    x1=df.iloc[0:n1,:p]
    x2=df.iloc[n2:len(df.index),:p]
    x2.reset_index(inplace = True, drop = True)
    x1_bar=mean_vector(x1)
    x2_bar=mean_vector(x2)
    S1=cov_matrix(x1)
    S2=cov_matrix(x2)
    Sp=((n1-1)*S1+(n2-1)*S2)/(n1+n2-2) #pooled covariance matrix
    t2 = n1*n2/(n1+n2)*(x1_bar-x2_bar).dot(np.linalg.inv(Sp)).dot((x1_bar-x2_bar).T).iloc[0,0] #test statistic
    fvalue = (n1+n2-p-1)/(p*(n1+n2-2))*t2
    pvalue = f.sf(fvalue, p, n1+n2-p-1)
    return {'t2': t2, 'f-value': fvalue, 'p-value': pvalue}

In [17]:
#3
#(a)
turtle = pd.read_fwf('turtle.dat', header=None, widths = [5,5,4,8])
turtle

,0,1,2,3
0,98,81,38,female
1,103,84,38,female
2,103,86,42,female
3,105,86,42,female
4,109,88,44,female
5,123,92,50,female
6,123,95,46,female
7,133,99,51,female
8,133,102,51,female
9,133,102,51,female


In [18]:
two_sample_hotelling(turtle)
#p-value < 0.05
#Thus, we reject the null hypothesis

{'t2': 72.38162304698841,
 'f-value': 23.07819865266297,
 'p-value': 3.9667264134848684e-09}

In [19]:
#(b) Determine lengths and directions for the axes of 95% confidence elipsoid
def conf_region(df,alpha):
    n1=int(len(df.index)/2) #number of observations for X1
    n2=n1
    p=len(df.columns)-1 #number of variables
    x1=df.iloc[0:n1,:p]
    x2=df.iloc[n2:len(df.index),:p]
    x2.reset_index(inplace = True, drop = True)
    S1=cov_matrix(x1)
    S2=cov_matrix(x2)
    Sp=((n1-1)*S1+(n2-1)*S2)/(n1+n2-2) #pooled covariance matrix
    eigenval = np.linalg.eig(Sp)[0] #eigenvalues
    eigenvec = np.linalg.eig(Sp)[1] #eigenvectors
    c2 = p*(n1+n2-2)/(n1+n2-p-1)*f.isf(alpha,p,n1+n2-p-1)
    halflengths = np.sqrt(eigenval)*np.sqrt((1/n1+1/n2)*c2)
    lengths = 2*halflengths
    return 'lengths:', lengths, 'directions for the axes:', eigenvec

In [20]:
conf_region(turtle,0.05)

('lengths:',
 array([35.84469186,  3.92249939,  2.68457971]),
 'directions for the axes:',
 array([[-0.82017422, -0.54070785, -0.18694725],
        [-0.49543231,  0.83467139, -0.24056286],
        [-0.28611374,  0.10468375,  0.9524601 ]]))

In [21]:
#(c) Find 95% simultaneous confidence intervals
def sim_conf_int(df,alpha):
    n1=int(len(df.index)/2) #number of observations for X1
    n2=n1
    p=len(df.columns)-1 #number of variables
    x1=df.iloc[0:n1,:p]
    x2=df.iloc[n2:len(df.index),:p]
    x2.reset_index(inplace = True, drop = True)
    x1_bar=mean_vector(x1)
    x2_bar=mean_vector(x2)
    diff=x1_bar-x2_bar
    S1=cov_matrix(x1)
    S2=cov_matrix(x2)
    Sp=((n1-1)*S1+(n2-1)*S2)/(n1+n2-2) #pooled covariance matrix
    c2 = p*(n1+n2-2)/(n1+n2-p-1)*f.isf(alpha,p,n1+n2-p-1)
    
    a1=pd.DataFrame([1,0,0], index=diff.columns, columns=['mean'])
    a2=pd.DataFrame([0,1,0], index=diff.columns, columns=['mean'])
    a3=pd.DataFrame([0,0,1], index=diff.columns, columns=['mean'])
    a=[a1,a2,a3]    
    conf_int=[]
    
    for i in range(p):
        term1=a[i].T.dot(diff.T).iloc[0,0]
        term2=np.sqrt(c2*(1/n1+1/n2)*a[i].T.dot(Sp).dot(a[i])).iloc[0,0]
        interval=[term1-term2, term1+term2]
        conf_int.append(interval)

    return conf_int

In [22]:
sim_conf_int(turtle,0.05)

[[7.92688151649465, 37.40645181683867],
 [5.256946590448267, 23.32638674288505],
 [6.044543977389379, 16.622122689277276]]

In [23]:
#(d) Find 95% Bonferroni confidence intervals
def bonferroni(df,alpha):
    n1=int(len(df.index)/2) #number of observations for X1
    n2=n1
    p=len(df.columns)-1 #number of variables
    x1=df.iloc[0:n1,:p]
    x2=df.iloc[n2:len(df.index),:p]
    x2.reset_index(inplace = True, drop = True)
    x1_bar=mean_vector(x1)
    x2_bar=mean_vector(x2)
    diff=x1_bar-x2_bar
    S1=cov_matrix(x1)
    S2=cov_matrix(x2)
    Sp=((n1-1)*S1+(n2-1)*S2)/(n1+n2-2) #pooled covariance matrix
       
    conf_int=[]
    
    for i in range(p):
        term1=diff.iloc[0,i]
        term2=t.isf(alpha/(2*p),n1+n2-2)*np.sqrt((1/n1+1/n2)*Sp.iloc[i,i])
        interval=[term1-term2, term1+term2]
        conf_int.append(interval)

    return conf_int

In [24]:
bonferroni(turtle,0.05)
#bonferroni intervals are more accurate than regular simultaneous confidence intervals

[[10.344184867410606, 34.98914846592271],
 [6.738627557458531, 21.844705775874782],
 [6.9118977482998565, 15.7547689183668]]

In [25]:
#(e) Test equality of the two population covariance matrices using your own code
def cov_equal(df):
    n1=int(len(df.index)/2) #number of observations for X1
    n2=n1
    p=len(df.columns)-1 #number of variables
    x1=df.iloc[0:n1,:p]
    x2=df.iloc[n2:len(df.index),:p]
    x2.reset_index(inplace = True, drop = True)
    S1=cov_matrix(x1)
    S2=cov_matrix(x2)
    Sp=((n1-1)*S1+(n2-1)*S2)/(n1+n2-2) #pooled covariance matrix
    
    M=(n1+n2-2)*np.log(np.linalg.det(Sp))-(n1-1)*np.log(np.linalg.det(S1))-(n2-1)*np.log(np.linalg.det(S2)) #test statistic
    C_inv = 1-(2*p**2+3*p-1)/(6*(p+1))*((n1+n2-2)/((n1-1)*(n2-1))-1/(n1+n2-2)) #scale factor
    MC=M*C_inv
    
    pvalue = chi2.sf(MC, p*(p+1)/2)
    
    return {'MC^(-1)': MC, 'p-value': pvalue}

In [26]:
cov_equal(turtle)
#At 0.05 significance level, p-value=0.000671<0.05
#Thus, we reject the null hypothesis

{'MC^(-1)': 23.404913718644007, 'p-value': 0.000671608688820075}

In [27]:
#(f) Test equality of the two population covariance matrices using Python package
x1=turtle.iloc[0:24,:3]
x2=turtle.iloc[24:48,:3]
x2.reset_index(inplace = True, drop = True)
S1=cov_matrix(x1)
S2=cov_matrix(x2)

mv.test_cov_oneway([S1,S2],[24,24])
#same result

<class 'statsmodels.stats.base.HolderTuple'>
statistic = 3.8991762550673945
pvalue = 0.0006786237151972158
statistic_base = 25.18423464462279
statistic_chi2 = 23.404913718644007
pvalue_chi2 = 0.000671608688820075
df_chi2 = 6.0
distr_chi2 = 'chi2'
statistic_f = 3.8991762550673945
pvalue_f = 0.0006786237151972158
df_f = (6.0, 15331.018867924493)
distr_f = 'F'
tuple = (3.8991762550673945, 0.0006786237151972158)